# Create 2D representation of trees based on a given distance

### Boilerplate database setup

In [1]:
import os, sys
sys.path.insert(1, os.path.join(os.path.abspath(os.curdir), 'spreadr'))

import django
from django.conf import settings
from spreadr import settings as base_spreadr_settings
spreadr_settings = base_spreadr_settings.__dict__.copy()
spreadr_settings['DATABASES'] = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        'NAME': 'spreadr_exp_1',
        'USER': 'spreadr_analysis'
    }
}
settings.configure(**spreadr_settings)
django.setup()